### Spark fundamentals

In [1]:
# Import pyspark(Spark's python wrapper)
from pyspark.sql import SparkSession

In [2]:
# Create spark session
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/20 09:37:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Define file path
file_path = "./data/people.csv"

# Load data from a csv file
df = spark.read.csv(file_path, header=True)

In [4]:
# Define table name
table_name = "organizations"

# Create an SQL table 
df.createOrReplaceTempView(table_name)

In [5]:
# Define query
query = f"SELECT * FROM {table_name}"

# Query the table
spark.sql(query).show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+
|    1|E84A904909dF528|          Liu-Hoover|http://www.day-ha...|      Western Sahara|Ergonomic zero ad...|   1980|   Online Publishing|               6852|
|    2|AAC4f9aBF86EAeF|       Orr-Armstrong|https://www.chapm...|             Algeria|Ergonomic radical...|   1970|     Import / Export|               7994|
|    3|ad2eb3C8C24DB87|           Gill-Lamb|     http://lin.com/|       Cote d'Ivoire|Programmable inte...|   2005|   Apparel / Fashion|               5105|
|    4|D76BB12E5eE165B|         Bauer-Weiss|https://gilles

In [14]:
# Inspect table schema
result = spark.sql(f"SHOW COLUMNS FROM {table_name}")

result

DataFrame[col_name: string]

In [10]:
# Inspect table schema
result = spark.sql(f"SELECT * FROM {table_name} LIMIT 0")

result

DataFrame[Index: string, Organization Id: string, Name: string, Website: string, Country: string, Description: string, Founded: string, Industry: string, Number of employees: string]